<a href="https://colab.research.google.com/github/Avadhak47/Avadhak47/blob/main/notebooks/Deep%20Learning%20Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Human Mood recognition model

* dataset : fer2013
* classifier: EDNN model for FER by Deepak Kumar Jaina, Pourya Shamsolmoalib &
    Paramjit Sehdev, as it appears in "Extended deep neural network for 
    facial emotion recognition", 2019 

In [ ]:
# installing tensorflow
!pip install -q tensorflow==2.0.0 tensorboard==2.0.0

In [ ]:
import tensorflow as tf
tf.__version__

## The dataset loader

This function will allow us to load directly all the dataset using pandas. This will also split the dataset into training and testing subsets.

In [ ]:
from tensorflow import keras
import pandas as pd
import numpy as np


def load_dataset(net=True):
    """Utility function to load the FER2013 dataset.
    
    It returns the formated tuples (X_train, y_train) , (X_test, y_test).
    """

    # Load and filter in Training/not Training data:
    df = pd.read_csv('../data/fer2013.csv')
    training = df.loc[df['Usage'] == 'Training']
    testing = df.loc[df['Usage'] != 'Training']

    # X_train values:
    X_train = training[['pixels']].values
    X_train = [np.fromstring(e[0], dtype=int, sep=' ') for e in X_train]

    """
    Parameters
    ==========
    net : boolean
        This parameter is used to reshape the data from images in 
        (cols, rows, channels) format. In case that it is False, a standard
        format (cols, rows) is used.
    """
    if net:
        X_train = [e.reshape((48, 48, 1)).astype('float32') for e in X_train]
    else:
        X_train = [e.reshape((48, 48)) for e in X_train]
    X_train = np.array(X_train)

    # X_test values:
    X_test = testing[['pixels']].values
    X_test = [np.fromstring(e[0], dtype=int, sep=' ') for e in X_test]
    if net:
        X_test = [e.reshape((48, 48, 1)).astype('float32') for e in X_test]
    else:
        X_test = [e.reshape((48, 48)) for e in X_test]
    X_test = np.array(X_test)

    # y_train values:
    y_train = training[['emotion']].values
    y_train = keras.utils.to_categorical(y_train)

    # y_test values
    y_test = testing[['emotion']].values
    y_test = keras.utils.to_categorical(y_test)

    return (X_train, y_train) , (X_test, y_test)

We can now use our main function to load the complete dataset.

In [ ]:
(X_train, y_train) , (X_test, y_test) = load_dataset()

We can access the data and plot some samples to check out waht's inside:

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')

plt.figure(figsize=(12, 12))
for i in range(16):
    plt.subplot(4, 4, i + 1)
    plt.imshow(X_train[i].reshape((48, 48)), cmap="gray")
    plt.axis('off')
    plt.tight_layout()

And, to understand how each image is loaded, we can print any element as a matrix:

In [ ]:
X_train[i].reshape((48, 48))

We can now proceed to build our EDNN model:

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.optimizers import SGD


def ResidualBlock(prev_layer):
    """Residual block from the EDNN model for FER by Deepak Kumar Jaina,
    Pourya Shamsolmoalib & Paramjit Sehdev, as it appears in "Extended 
    deep neural network for facial emotion recognition", 2019.
    """
    conv_1 = Conv2D(64, (1, 1))(prev_layer)
    conv_2 = Conv2D(64, (3, 3), padding="same")(conv_1)
    shortc = concatenate([conv_1, conv_2], axis=-1)
    conv_3 = Conv2D(128, (3, 3), padding="same")(shortc)
    conv_4 = Conv2D(256, (1, 1))(conv_3)
    output = concatenate([conv_4, prev_layer], axis=-1)
    
    return output


def EDNN(n_classes=7):
    """
    EDNN model for FER by Deepak Kumar Jaina, Pourya Shamsolmoalib &
    Paramjit Sehdev, as it appears in "Extended deep neural network for 
    facial emotion recognition", 2019.
    """

    x = Input(shape=(48, 48, 1))
    y = Conv2D(32, (5, 5), input_shape=(48, 48, 1), strides=(2, 2), 
               data_format='channels_last')(x)
    y = MaxPooling2D(pool_size=(2, 2))(y)
    y = Conv2D(64, (3, 3), strides=(1, 1))(y)
    y = ResidualBlock(y)
    y = Conv2D(128, (3, 3), strides=(1, 1), padding="same")(y)
    y = MaxPooling2D(pool_size=(2, 2))(y)
    y = Conv2D(128, (3, 3), strides=(1, 1))(y)
    y = ResidualBlock(y)
    y = Conv2D(256, (3, 3), strides=(1, 1), padding="same")(y)
    y = MaxPooling2D(pool_size=(2, 2))(y)
    y = Conv2D(512, (3, 3), strides=(1, 1), padding="same")(y)
    y = Flatten()(y)
    y = Dense(1024, activation='relu')(y)
    y = Dropout(0.2)(y)
    y = Dense(512, activation='relu')(y)
    y = Dropout(0.2)(y)
    y = Dense(n_classes, activation='softmax')(y)
    
    # Create model:
    model = Model(x, y)

    # Compile model:
    opt = SGD(lr=LRATE, momentum=0.9, decay=LRATE/EPOCHS)
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt, metrics=['accuracy'])

    return model

We now create an instance of our model and verify the details of the architecture:

In [ ]:
# Set hyperparameters:
EPOCHS = 1
BATCH = 64
LRATE = 1e-4

# Instance model
ednn = EDNN()

In [ ]:
ednn.summary()

## Model training

So far we have created out model and we already have loaded the datset. But beofr, let's create our media folder:

In [ ]:
!mkdir ../media

We can now proceed to train the model.

In [ ]:
history = ednn.fit(X_train, y_train,
                   validation_data=(X_test, y_test),
                   epochs=EPOCHS, batch_size=BATCH)

After the training, we can plot the history of this process.

The following functions will allow us to do so.

In [ ]:
def plot_loss(history):
    plt.style.use("ggplot")
    plt.figure(figsize=(8, 4))
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title("Model's training loss")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss")
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()


def plot_accuracy(history):
    plt.style.use("ggplot")
    plt.figure(figsize=(8, 4))
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title("Model's training accuracy")
    plt.xlabel("Epoch #")
    plt.ylabel("Accuracy")
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

In [ ]:
# Plot loss:
plot_loss(history)
# plt.savefig('../media/loss.png', dpi=300)

# Plot accuracy:
plot_accuracy(history)
# plt.savefig('../media/accuracy.png', dpi=300)

A way to explore accuracy through classes is using a confusion matrix:

In [ ]:
# Create emotions map:
emotion_labels = [
    'Angry',
    'Disgust',
    'Fear',
    'Happy',
    'Sad',
    'Surprise',
    'Neutral'
]

# Predict using trained model:
y_pred = ednn.predict(X_test)
y_pred = np.asarray([np.argmax(e) for e in y_pred])
y_true = np.asarray([np.argmax(e) for e in y_test])

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns


# Compute confusion matrix:
cm = confusion_matrix(y_true, y_pred)
cm_normalised = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

# Plot confusion matrix:
sns.set(font_scale=1.5) 
fig, ax = plt.subplots(figsize=(10,10))
ax = sns.heatmap(cm_normalised, annot=True, linewidths=0, square=False, 
                 cmap='gray', yticklabels=emotion_labels,
                 xticklabels=emotion_labels, vmin=0,
                 vmax=np.max(cm_normalised), fmt=".2f",
                 annot_kws={"size": 20})
ax.set(xlabel='Predicted label', ylabel='True label')

## Saving the trained model

To save the trained model we will basically do two things:

- Serialize the model into a JSON file, which will save the architecture of our model.
- Serialize the weights into a HDF5 file, which will save all parameters of our model.

In [ ]:
# Serialize model to JSON:
json_ednn = ednn.to_json()
with open('../data/model.json', 'w') as json_file:
    json_file.write(json_ednn)

# Serialize weights to HDF5 (h5py needed):
ednn.save_weights('../data/model.h5')
print('Model saved to disk.')